In [1]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

In [3]:
df = pd.read_csv('C:/rsource2/tour_data.csv')
df1 = df.loc[:,['tour_id','title']] 

In [4]:
df1.iloc[200:240]

,tour_id,title
200,201,구월산
201,202,구의 아리수 정수센터
202,203,구정칼국수
203,204,국기원(세계태권도본부)
204,205,국립4.19민주묘지
205,206,국립고궁박물관
206,207,국립공원 산악박물관
207,208,국립국악박물관
208,209,국립국악원
209,210,국립극장


In [5]:
title = []
comments = []
ratings = []

In [6]:
def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    for i in range(200,240):
        #delapy
        sleep(3)
        print('####',i)
        search(df1['title'][i])

    driver.quit()
    print("finish")

In [7]:
def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    try:
        soup = BeautifulSoup(html, 'html.parser')
        place= soup.select('.placelist > .PlaceItem')[0] # 검색된 장소 목록
        try:
            sleep(1)
            crawling1(place)

        except ElementNotInteractableException:
            print('not found')
        finally:
            search_area.clear()
    except:
        print('no search result')

In [8]:
def crawling(place):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i in range(2):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review1(place_name)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

In [9]:
def crawling1(place):
        """
        페이지 목록을 받아서 크롤링 하는 함수
        :param place: 리뷰 정보 찾을 장소이름
        """

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review1(place_name)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

In [10]:
def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


In [11]:
def extract_review1(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            title.append(place_name)
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                
                comments.append(val)
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret

In [12]:
if __name__ == "__main__":
    title = []
    comments = []
    ratings = []
    main()

#### 200
#### 황해도구월산
여기 제육쌈밥 진짜 맛있어요  고기도 잡내가 없고 정말 깔끔해요 강추!/5
조미료 하나도 안쓰고 엄청나게 깔끔한맛! 여기서 밥먹음 정말 집밥 먹은것 같고 속도 편안하고 완전 좋음/5
도가니 수육, 담백하고 식감이 좋다. 주인 아저씨의 너스레는 서비스!/0
#### 201
#### 구의아리수정수센터앞 대여소
no review in extract
#### 202
#### 구정만두칼국수
칼국수에서 쇳조각 나옴. /1
맛은 그냥그런데 서비스가 엉망 처음부터 끝까지 반말하며 안좋은 표정으로 주문받는 완전 불친절한 종업원.. ㅋ 다신 안갈듯 ㅋ ;; /1
청국장 한 입 먹고 미소가 절로 나왔어요 열무 보리비빔밥에 청국장 한 숟가락 넣고 비벼 먹는데 이것도 맛있었어요 혼자 갔었는데 친한친구랑 같이 또 싶어진 곳이에요/5
#### 203
no search result
#### 204
no search result
#### 205
no search result
#### 206
no search result
#### 207
no search result
#### 208
no search result
#### 209
no search result
#### 210
no search result
#### 211
no search result
#### 212
no search result
#### 213
no search result
#### 214
no search result
#### 215
no search result
#### 216
no search result
#### 217
no search result
#### 218
no search result
#### 219
no search result
#### 220
no search result
#### 221
no search result
#### 222
no search result
#### 223
no search result
#### 224
no search result
#### 2

In [13]:
title

['황해도구월산', '황해도구월산', '황해도구월산', '구정만두칼국수', '구정만두칼국수', '구정만두칼국수']

In [14]:
len(title)

6

In [15]:
comments

['여기 제육쌈밥 진짜 맛있어요  고기도 잡내가 없고 정말 깔끔해요 강추!/5',
 '조미료 하나도 안쓰고 엄청나게 깔끔한맛! 여기서 밥먹음 정말 집밥 먹은것 같고 속도 편안하고 완전 좋음/5',
 '도가니 수육, 담백하고 식감이 좋다. 주인 아저씨의 너스레는 서비스!/0',
 '칼국수에서 쇳조각 나옴. /1',
 '맛은 그냥그런데 서비스가 엉망 처음부터 끝까지 반말하며 안좋은 표정으로 주문받는 완전 불친절한 종업원.. ㅋ 다신 안갈듯 ㅋ ;; /1',
 '청국장 한 입 먹고 미소가 절로 나왔어요 열무 보리비빔밥에 청국장 한 숟가락 넣고 비벼 먹는데 이것도 맛있었어요 혼자 갔었는데 친한친구랑 같이 또 싶어진 곳이에요/5']

In [16]:
for i in range(len(comments)):
    ratings.append(str(comments[i])[-1])

In [17]:
temp1 = pd.DataFrame({'title':title,'comments':comments,'ratings':ratings})

In [24]:
str(comments[2])[-1]

'5'

In [32]:
temp1

,title,comments,ratings
0,가마목,최고예요,요
1,가마목,최고예요,요
2,가마목,최고예요/5,5
3,가마목,점심에 쌈밥 짱맛있어용 볶음밥도 꿀맛/5,5
4,가마목,삼겹살 너무 맛있고 직접 고기도 구워주시고 정말 친절한 식당입니다!/5,5
5,가마목,주문하면 화내는집. 고기주문할려고 벨 누를때마다 아주머니가 한숨쉬면서 짜증내면서 오...,1
6,가마목,"별 1점도 아까운집 가격, 고기, 서비스 뭐 하나라도 조금이라도 좋은게 없는집/1",1


In [33]:
ratings

['요', '요', '5', '5', '5', '1', '1']

In [34]:
comments

['최고예요',
 '최고예요',
 '최고예요/5',
 '점심에 쌈밥 짱맛있어용 볶음밥도 꿀맛/5',
 '삼겹살 너무 맛있고 직접 고기도 구워주시고 정말 친절한 식당입니다!/5',
 '주문하면 화내는집. 고기주문할려고 벨 누를때마다 아주머니가 한숨쉬면서 짜증내면서 오심 ㅋㅋㅋ 그런식으로 할꺼면 장사하지마세요/1',
 '별 1점도 아까운집 가격, 고기, 서비스 뭐 하나라도 조금이라도 좋은게 없는집/1']

In [35]:
len(comments)

7

In [36]:
len(title)

7

In [37]:
title

['가마목', '가마목', '가마목', '가마목', '가마목', '가마목', '가마목']